### Pets Dataset

In [ ]:
import os, time, pickle, json
import numpy as np
import pandas as pd
from dotenv import load_dotenv; load_dotenv()
from modules.prompts import DefaultPrompt

DefPrompt = DefaultPrompt("Defaut")

images_path = "../database/Pets/images/" # Folder with images
labels_path = "../data/pets/test_stratified_subset_30.csv" # File with labels
working_file = "../data/pets/test_stratified_subset_5.csv" # File with working files info
image_features = "../data/pets/openai/image_features.parquet" # File to save the output image features
labels_features = "../data/pets/openai/label_features.parquet" # File to save the output labels features

from modules.llm import OpenAIModel, GeminiModel, LlavaModel
from dotenv import load_dotenv; load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OpenAI = OpenAIModel(OPENAI_API_KEY, model_name='gpt-4.1-nano-2025-04-14')

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
Gemini = GeminiModel(GEMINI_API_KEY, model_name='gemini-1.5-flash-8b-001')

REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")
LLaVA = LlavaModel(REPLICATE_API_TOKEN)

In [ ]:
class Pets:
    def __init__(self, images_path, class_meta_path):
        self.meta = {**json.load(open(class_meta_path, "r"))}
        self.images_path = images_path
        self.classes = list(self.meta['labels'].keys())

    def json_to_df(self, json_data):
        # Convert the classes dictionary to a DataFrame
        df = pd.DataFrame.from_dict(json_data, orient='index').reset_index()
        df = df.rename(columns={"index": "name"})
        return df.sort_values(by="name").reset_index(drop=True)

# --------------------------------------------------------------------------
pets_images_path = "../database/Pets/images/" # Folder with images
pets_class_meta_path = "../data/pets/pets_meta.json"
Pet = Pets(pets_images_path, pets_class_meta_path)

In [ ]:
from modules.gen_features import ImageFeatures
pets_image_working_file = "../data/pets/test/image_features.parquet"
PetsFeatures = ImageFeatures(pets_images_path, pets_class_meta_path, 
                             pets_image_working_file, llm_model=OpenAI)
# PetsFeatures.gen_info()
PetsFeatures.img_features.head(3)

In [ ]:
from modules.gen_features import LabelFeatures
pets_labels_working_file = "../data/pets/test/label_features.parquet" # File to save the output labels features
PetsLabelFeatures = LabelFeatures(pets_images_path, pets_class_meta_path, 
                                  pets_labels_working_file, llm_model=OpenAI)
PetsLabelFeatures.label_features.head(3)

In [ ]:
from modules.prompts import DefaultPrompt
from modules.encoder import FeaturesEncoder

model = "../models/jina-clip-v1"
images_path = "../database/Pets/images/"

img_features_path = "../data/pets/test/image_features.parquet" 
labels_features_path = "../data/pets/test/label_features.parquet"

encoding_images_path = "../data/pets/test/image_features.pkl"
encoding_labels_path  = "../data/pets/test/label_features.pkl"

FE = FeaturesEncoder(images_path, encoding_images_path, encoding_labels_path, img_file_type='jpg', model=model)

# FE.encode_images(img_features_path)
DefPrompt = DefaultPrompt('Default')
human_design_prompt = "A photo of {}, a type of pets"
FE.encode_labels(labels_features_path, human_design_prompt)


### Prediction

#### 1. Test

In [31]:
from modules.classifier import ImageClassifier
# Classification
encoded_image_file = "../data/pets/test/image_features.pkl"
encoded_text_file  = "../data/pets/test/label_features.pkl"

with open(encoded_image_file, "rb") as f: 
    img_features = pickle.load(f)

with open(encoded_text_file, "rb") as f: 
    label_features = pickle.load(f)

I4P = ImageClassifier(label_features, mode='M3', img_features=img_features, ifeature='X_q')

print("="*50)
df = I4P.classify()
accuracy, precision, recall, f1 = I4P.evaluation(df)

Using model M3: Class Description Features Embedding
Using Image Feature: Encoded Fused Image Feature X_q
Accuracy: 0.8541
Precision: 0.8541
Recall: 0.8343
F1-score: 0.8341


#### 2. OpenAI

In [2]:
import pickle
from modules.classifier import ImageClassifier
# Classification
encoded_image_file = "../data/pets/openai/image_features.pkl"
# encoded_text_file  = "../data/pets/openai/label_features.pkl"
encoded_text_file  = "../data/pets/openai/label_full_features.pkl"

with open(encoded_image_file, "rb") as f: 
    img_features = pickle.load(f)

with open(encoded_text_file, "rb") as f: 
    label_features = pickle.load(f)

import pandas as pd
acc_df = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1'])
for X in ['X_if', 'X_df', 'X_pf', 'X_q']:
    I4P = ImageClassifier(label_features, mode='M4', img_features=img_features, ifeature=X)
    print("="*50)
    df = I4P.classify()
    accuracy, precision, recall, f1 = I4P.evaluation(df)
    acc_df.loc[X] = [accuracy, precision, recall, f1]

save_path = "../data/accuracies/pet_openai_default.csv"
acc_df.to_csv(save_path, index=False)
acc_df.head()

Using model M4: Fused Features Embedding
Using Image Feature: Encoded Image X_if
Accuracy: 0.7459
Precision: 0.7459
Recall: 0.7858
F1-score: 0.7356
Using model M4: Fused Features Embedding
Using Image Feature: Encoded Image Description X_df
Accuracy: 0.4973
Precision: 0.4973
Recall: 0.5496
F1-score: 0.4675
Using model M4: Fused Features Embedding
Using Image Feature: Encoded Init Prediction X_pf
Accuracy: 0.8378
Precision: 0.8378
Recall: 0.8027
F1-score: 0.8091
Using model M4: Fused Features Embedding
Using Image Feature: Encoded Fused Image Feature X_q
Accuracy: 0.8432
Precision: 0.8432
Recall: 0.8072
F1-score: 0.8153


,accuracy,precision,recall,f1
X_if,0.745946,0.745946,0.785848,0.735589
X_df,0.497297,0.497297,0.549580,0.467501
X_pf,0.837838,0.837838,0.802729,0.809099
X_q,0.843243,0.843243,0.807234,0.815310


#### 3. LLaVA

In [3]:
from modules.classifier import ImageClassifier
# Classification
encoded_image_file = "../data/pets/llava/image_features.pkl"
encoded_text_file  = "../data/pets/llava/label_features.pkl"

with open(encoded_image_file, "rb") as f: 
    img_features = pickle.load(f)

with open(encoded_text_file, "rb") as f: 
    label_features = pickle.load(f)

import pandas as pd
acc_df = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1'])
for X in ['X_if', 'X_df', 'X_pf', 'X_q']:
    I4P = ImageClassifier(label_features, mode='M4', img_features=img_features, ifeature=X)
    print("="*50)
    df = I4P.classify()
    accuracy, precision, recall, f1 = I4P.evaluation(df)
    acc_df.loc[X] = [accuracy, precision, recall, f1]

save_path = "../data/accuracies/pet_llava_default.csv"
acc_df.to_csv(save_path, index=False)
acc_df.head()

Using model M4: Fused Features Embedding
Using Image Feature: Encoded Image X_if
Accuracy: 0.7351
Precision: 0.7351
Recall: 0.7848
F1-score: 0.7230
Using model M4: Fused Features Embedding
Using Image Feature: Encoded Image Description X_df
Accuracy: 0.2108
Precision: 0.2108
Recall: 0.2100
F1-score: 0.1817
Using model M4: Fused Features Embedding
Using Image Feature: Encoded Init Prediction X_pf
Accuracy: 0.3243
Precision: 0.3243
Recall: 0.3238
F1-score: 0.2709
Using model M4: Fused Features Embedding
Using Image Feature: Encoded Fused Image Feature X_q
Accuracy: 0.3297
Precision: 0.3297
Recall: 0.3262
F1-score: 0.2749


,accuracy,precision,recall,f1
X_if,0.735135,0.735135,0.784815,0.723033
X_df,0.210811,0.210811,0.209978,0.181742
X_pf,0.324324,0.324324,0.323801,0.270898
X_q,0.329730,0.329730,0.326150,0.274886


### Backup

In [ ]:
img_pkl_path = "../data/pets/openai/image_features.pkl"
with open(img_pkl_path, "rb") as f:
    img_features = pickle.load(f)
img_features['Abyssinian_1'].keys()

labels_features = "../data/pets/llava/label_features.parquet"
label_features_df = pd.read_parquet(labels_features)
print(len(label_features_df))
label_features_df.head(3)

working_file = "../data/pets/openai/image_features.parquet"
img_features_df = pd.read_parquet(working_file)
print(len(img_features_df))
img_features_df.tail(3)

# Accuracy
img_features_df['init_pred'] = img_features_df['init_pred'].str.lower()
img_features_df['label_id'] = img_features_df['label_id'].str.lower()

count = 0
for i in range(len(img_features_df)):
    if img_features_df['init_pred'][i] == img_features_df['label_id'][i]:
        count += 1

print(count/len(img_features_df))

# Display Image 
from PIL import Image
import matplotlib.pyplot as plt
from pprint import pprint
def display_image(id):
    pprint(f"Description: {img_features_df['img_desc'][id]}")
    print("Initial Prediction: ", img_features_df['init_pred'][id])
    print("True Label: ", img_features_df['label_id'][id])
    img_path = '../database/Pets/images/'
    image_path = os.path.join(img_path, f"{img_features_df['filename'][id]}.jpg")
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    print("="*50)


for i in range(10):
    display_image(i)

### Pets Processed data

#### 1. Test

In [2]:
import pickle
import pandas as pd

image_features = "../data/pets/test/image_features.parquet" # File to save the output image features
labels_features = "../data/pets/test/label_features.parquet"

encoded_image_file = "../data/pets/test/image_features.pkl"
encoded_text_file  = "../data/pets/test/label_features.pkl"

image_features_par = pd.read_parquet(image_features)
display(image_features_par.head(3))
labels_features_par = pd.read_parquet(labels_features)
display(labels_features_par.head(1))

with open(encoded_image_file, "rb") as f: 
    img_features_pkl = pickle.load(f)
print(img_features_pkl.keys())
print(img_features_pkl['Abyssinian_1'].keys())

with open(encoded_text_file, "rb") as f: 
    label_features_pkl = pickle.load(f)
print(label_features_pkl.keys())
print(label_features_pkl['Persian'].keys())


,file_name,label_id,init_pred,img_desc
0,Abyssinian_1,Abyssinian,Bengal,The image shows a cat with the body of an Abys...
1,Abyssinian_94,Abyssinian,Egyptian Mau,The image shows a person wearing glasses and a...
2,Abyssinian_106,Abyssinian,Egyptian Mau,The image shows a domestic cat lying on a shin...


,label,CDR_10,CDR_20,CDR_30,CDR_40,CDR_50,CDR_11,CDR_21,CDR_31,CDR_41,...,CDR_18,CDR_28,CDR_38,CDR_48,CDR_58,CDR_19,CDR_29,CDR_39,CDR_49,CDR_59
0,British Shorthair,"The British Shorthair is a sturdy, muscular ca...",A British Shorthair is characterized by its ro...,"The British Shorthair has a round face, large,...","The image shows a plush, round British Shortha...","A charming British Shorthair with a plush, rou...","The British Shorthair is a plush, muscular cat...",A British Shorthair is characterized by its ro...,"The British Shorthair has a round face, large ...",The image depicts a charming British Shorthair...,...,"The British Shorthair is a sturdy, round-faced...",A British Shorthair is characterized by its ro...,"The British Shorthair is a medium to large, we...",The image depicts a plush British Shorthair ca...,A fluffy British Shorthair radiates charm and ...,"The British Shorthair is a round-faced, muscul...",A British Shorthair is distinguished by its ro...,"The British Shorthair is a medium to large, mu...",The image shows a plush British Shorthair with...,A adorable British Shorthair with lush gray fu...


dict_keys(['Abyssinian_1', 'Abyssinian_94', 'Abyssinian_106', 'Abyssinian_57', 'Abyssinian_6', 'Bengal_13', 'Bengal_94', 'Bengal_50', 'Bengal_180', 'Bengal_192', 'Birman_13', 'Birman_94', 'Birman_50', 'Birman_180', 'Birman_191', 'Bombay_104', 'Bombay_3', 'Bombay_25', 'Bombay_24', 'Bombay_59', 'British_Shorthair_149', 'British_Shorthair_94', 'British_Shorthair_43', 'British_Shorthair_196', 'British_Shorthair_212', 'Egyptian_Mau_207', 'Egyptian_Mau_130', 'Egyptian_Mau_4', 'Egyptian_Mau_104', 'Egyptian_Mau_222', 'Maine_Coon_148', 'Maine_Coon_94', 'Maine_Coon_41', 'Maine_Coon_203', 'Maine_Coon_21', 'Persian_149', 'Persian_91', 'Persian_44', 'Persian_195', 'Persian_21', 'Ragdoll_146', 'Ragdoll_94', 'Ragdoll_44', 'Ragdoll_191', 'Ragdoll_203', 'Russian_Blue_146', 'Russian_Blue_94', 'Russian_Blue_41', 'Russian_Blue_191', 'Russian_Blue_201', 'Siamese_150', 'Siamese_94', 'Siamese_47', 'Siamese_198', 'Siamese_20', 'Sphynx_152', 'Sphynx_93', 'Sphynx_38', 'Sphynx_197', 'Sphynx_20', 'american_bulldo

#### 2. OpenAI

In [ ]:
import pickle
import pandas as pd

image_features = "../data/pets/openai/image_features.parquet"
labels_features = "../data/pets/openai/label_features.parquet"

encoded_image_file = "../data/pets/openai/image_features.pkl"
encoded_text_file  = "../data/pets/openai/label_features.pkl"
# encoded_text_file_2  = "../data/pets/openai/label_full_features.pkl"

image_features_par = pd.read_parquet(image_features)
display(image_features_par.head(3))

labels_features_par = pd.read_parquet(labels_features)
display(labels_features_par.head(1))

with open(encoded_image_file, "rb") as f: 
    img_features_pkl = pickle.load(f)
print(img_features_pkl.keys())
print(img_features_pkl['Abyssinian_1'].keys())

with open(encoded_text_file, "rb") as f: 
    label_features_pkl = pickle.load(f)
print(label_features_pkl.keys())
print(label_features_pkl['Persian'].keys())


,file_name,label_id,init_pred,img_desc
0,Abyssinian_1,Abyssinian,Bengal,The image shows a cat with the body of an Abys...
1,Abyssinian_94,Abyssinian,Egyptian Mau,The image shows a person wearing glasses and a...
2,Abyssinian_106,Abyssinian,Egyptian Mau,The image shows a domestic cat lying on a shin...


,label,CDR_10,CDR_20,CDR_30,CDR_40,CDR_50,CDR_11,CDR_21,CDR_31,CDR_41,...,CDR_18,CDR_28,CDR_38,CDR_48,CDR_58,CDR_19,CDR_29,CDR_39,CDR_49,CDR_59
0,British Shorthair,"The British Shorthair is a sturdy, muscular ca...",A British Shorthair is characterized by its ro...,"The British Shorthair has a round face, large,...","The image shows a plush, round British Shortha...","A charming British Shorthair with a plush, rou...","The British Shorthair is a plush, muscular cat...",A British Shorthair is characterized by its ro...,"The British Shorthair has a round face, large ...",The image depicts a charming British Shorthair...,...,"The British Shorthair is a sturdy, round-faced...",A British Shorthair is characterized by its ro...,"The British Shorthair is a medium to large, we...",The image depicts a plush British Shorthair ca...,A fluffy British Shorthair radiates charm and ...,"The British Shorthair is a round-faced, muscul...",A British Shorthair is distinguished by its ro...,"The British Shorthair is a medium to large, mu...",The image shows a plush British Shorthair with...,A adorable British Shorthair with lush gray fu...


dict_keys(['Abyssinian_1', 'Abyssinian_94', 'Abyssinian_106', 'Abyssinian_57', 'Abyssinian_6', 'Bengal_13', 'Bengal_94', 'Bengal_50', 'Bengal_180', 'Bengal_192', 'Birman_13', 'Birman_94', 'Birman_50', 'Birman_180', 'Birman_191', 'Bombay_104', 'Bombay_3', 'Bombay_25', 'Bombay_24', 'Bombay_59', 'British_Shorthair_149', 'British_Shorthair_94', 'British_Shorthair_43', 'British_Shorthair_196', 'British_Shorthair_212', 'Egyptian_Mau_207', 'Egyptian_Mau_130', 'Egyptian_Mau_4', 'Egyptian_Mau_104', 'Egyptian_Mau_222', 'Maine_Coon_148', 'Maine_Coon_94', 'Maine_Coon_41', 'Maine_Coon_203', 'Maine_Coon_21', 'Persian_149', 'Persian_91', 'Persian_44', 'Persian_195', 'Persian_21', 'Ragdoll_146', 'Ragdoll_94', 'Ragdoll_44', 'Ragdoll_191', 'Ragdoll_203', 'Russian_Blue_146', 'Russian_Blue_94', 'Russian_Blue_41', 'Russian_Blue_191', 'Russian_Blue_201', 'Siamese_150', 'Siamese_94', 'Siamese_47', 'Siamese_198', 'Siamese_20', 'Sphynx_152', 'Sphynx_93', 'Sphynx_38', 'Sphynx_197', 'Sphynx_20', 'american_bulldo

#### 3. LLaVA

In [8]:
import pickle
import pandas as pd

image_features = "../data/pets/llava/image_features.parquet"
labels_features = "../data/pets/llava/label_features.parquet"

encoded_image_file = "../data/pets/llava/image_features.pkl"
encoded_text_file  = "../data/pets/llava/label_features.pkl"

image_features_par = pd.read_parquet(image_features)
display(image_features_par.head(3))

labels_features_par = pd.read_parquet(labels_features)
display(labels_features_par.head(1))

with open(encoded_image_file, "rb") as f: 
    img_features_pkl = pickle.load(f)
print(img_features_pkl.keys())
print(img_features_pkl['Abyssinian_1'].keys())

with open(encoded_text_file, "rb") as f: 
    label_features_pkl = pickle.load(f)
print(label_features_pkl.keys())
print(label_features_pkl['Persian'].keys())


,file_name,label_id,init_pred,img_desc
0,Abyssinian_1,Abyssinian,Abyssinian,The image features a brown and white cat sitti...
1,Abyssinian_94,Abyssinian,Wheaten terrier,The image features a man wearing glasses and a...
2,Abyssinian_106,Abyssinian,Abyssinian,The image features a brown and black cat lying...


,label,CDR_11,CDR_21,CDR_31,CDR_41,CDR_51,CDR_12,CDR_22,CDR_32,CDR_42,CDR_52
0,Abyssinian,The Abyssinian cat is characterized by its sho...,"Abyssinians are distinguished by their short, ...","The Abyssinian cat is known for its short, tic...","The image features a sleek, medium-sized Abyss...","""Captivating elegance: the Abyssinian cat flau...",The Abyssinian cat is characterized by its sho...,An Abyssinian cat can be identified by its dis...,The Abyssinian cat has a distinctive short coa...,An Abyssinian cat is often depicted with its s...,"""Elegant and playful, the Abyssinian cat embod..."


dict_keys(['Abyssinian_1', 'Abyssinian_94', 'Abyssinian_106', 'Abyssinian_57', 'Abyssinian_6', 'Bengal_13', 'Bengal_94', 'Bengal_50', 'Bengal_180', 'Bengal_192', 'Birman_13', 'Birman_94', 'Birman_50', 'Birman_180', 'Birman_191', 'Bombay_104', 'Bombay_3', 'Bombay_25', 'Bombay_24', 'Bombay_59', 'British_Shorthair_149', 'British_Shorthair_94', 'British_Shorthair_43', 'British_Shorthair_196', 'British_Shorthair_212', 'Egyptian_Mau_207', 'Egyptian_Mau_130', 'Egyptian_Mau_4', 'Egyptian_Mau_104', 'Egyptian_Mau_222', 'Maine_Coon_148', 'Maine_Coon_94', 'Maine_Coon_41', 'Maine_Coon_203', 'Maine_Coon_21', 'Persian_149', 'Persian_91', 'Persian_44', 'Persian_195', 'Persian_21', 'Ragdoll_146', 'Ragdoll_94', 'Ragdoll_44', 'Ragdoll_191', 'Ragdoll_203', 'Russian_Blue_146', 'Russian_Blue_94', 'Russian_Blue_41', 'Russian_Blue_191', 'Russian_Blue_201', 'Siamese_150', 'Siamese_94', 'Siamese_47', 'Siamese_198', 'Siamese_20', 'Sphynx_152', 'Sphynx_93', 'Sphynx_38', 'Sphynx_197', 'Sphynx_20', 'american_bulldo

### ImageNet Processed data

In [ ]:
import pickle
import pandas as pd

image_features = "../data/ImageNet_np/image_features.parquet"
labels_features = "../data/ImageNet_np/label_features.parquet"

encoded_image_file = "../data/ImageNet_np/image_features.pkl"
encoded_text_file  = "../data/ImageNet_np/label_features.pkl"

image_features_par = pd.read_parquet(image_features)
display(image_features_par.head(3))

labels_features_par = pd.read_parquet(labels_features)
display(labels_features_par.head(1))

with open(encoded_image_file, "rb") as f: 
    img_features_pkl = pickle.load(f)
print(img_features_pkl.keys())
print(img_features_pkl['ILSVRC2012_val_00028983'].keys())

with open(encoded_text_file, "rb") as f: 
    label_features_pkl = pickle.load(f)
print(label_features_pkl.keys())
print(label_features_pkl['n01440764'].keys())

,file_name,label_id,init_pred,img_desc
0,ILSVRC2012_val_00028983,n03538406,horse,The image features a horse that appears to be ...
1,ILSVRC2012_val_00028481,n03196217,microwave,The image shows the display panel of a microwa...
2,ILSVRC2012_val_00022716,n03961711,grocery store,The image features a set of kitchen cabinets m...


,id,labels,CDR_11,CDR_21,CDR_31,CDR_41,CDR_51,CDR_12,CDR_22,CDR_32,CDR_42,CDR_52
0,n01440764,"[tench, tinca tinca]",The tench (*Tinca tinca*) is a freshwater fish...,The tench (Tinca tinca) can be identified by i...,The tench (Tinca tinca) is a freshwater fish c...,The image of a tench (Tinca tinca) typically f...,"""Majestic Tench (Tinca tinca): Nature's shimme...","The tench, scientifically known as *Tinca tinc...",A tench (*Tinca tinca*) can be identified by i...,"The tench, or Tinca tinca, is a freshwater fis...",The image of a tench (Tinca tinca) showcases i...,"""Meet the Tench: a resilient beauty of freshwa..."


dict_keys(['ILSVRC2012_val_00028983', 'ILSVRC2012_val_00028481', 'ILSVRC2012_val_00022716', 'ILSVRC2012_val_00024691', 'ILSVRC2012_val_00011415', 'ILSVRC2012_val_00012377', 'ILSVRC2012_val_00040413', 'ILSVRC2012_val_00001893', 'ILSVRC2012_val_00032755', 'ILSVRC2012_val_00007001', 'ILSVRC2012_val_00018074', 'ILSVRC2012_val_00002469', 'ILSVRC2012_val_00002744', 'ILSVRC2012_val_00040693', 'ILSVRC2012_val_00025062', 'ILSVRC2012_val_00036553', 'ILSVRC2012_val_00039539', 'ILSVRC2012_val_00047725', 'ILSVRC2012_val_00037332', 'ILSVRC2012_val_00013737', 'ILSVRC2012_val_00046346', 'ILSVRC2012_val_00040732', 'ILSVRC2012_val_00022017', 'ILSVRC2012_val_00004624', 'ILSVRC2012_val_00014292', 'ILSVRC2012_val_00008961', 'ILSVRC2012_val_00043366', 'ILSVRC2012_val_00029627', 'ILSVRC2012_val_00002006', 'ILSVRC2012_val_00001059', 'ILSVRC2012_val_00047000', 'ILSVRC2012_val_00028661', 'ILSVRC2012_val_00012673', 'ILSVRC2012_val_00019019', 'ILSVRC2012_val_00044199', 'ILSVRC2012_val_00001184', 'ILSVRC2012_val_0

In [29]:
from modules.classifier import ImageClassifier
# Classification
encoded_image_file = "../data/ImageNet_np/image_features.pkl"
encoded_text_file  = "../data/ImageNet_np/label_features.pkl"

with open(encoded_image_file, "rb") as f: 
    img_features = pickle.load(f)

with open(encoded_text_file, "rb") as f: 
    label_features = pickle.load(f)

I4P = ImageClassifier(label_features, mode='M4', img_features=img_features, ifeature='X_q')

print("="*50)
df = I4P.classify()
accuracy, precision, recall, f1 = I4P.evaluation(df)

Using model M4: Fused Features Embedding
Using Image Feature: Encoded Fused Image Feature X_q
Accuracy: 0.4350
Precision: 0.4350
Recall: 0.4373
F1-score: 0.4270
